# Using dill to pickle anything

ipyparallel doesn't do much in the way of serialization.
It has custom zero-copy handling of numpy arrays,
but other than that, it doesn't do anything other than the bare minimum to make basic interactively defined functions and classes sendable.

There are a few projects that extend pickle to make just about anything sendable, and one of these is [dill](https://dill.readthedocs.io).
Another is [cloudpickle](https://github.com/cloudpipe/cloudpickle).

To install dill:
        
    pip install dill

First, as always, we create a task function, this time with a closure

In [1]:
def make_closure(a):
    """make a weird function with a closure on an open file, and return it"""
    import os

    f = open('/tmp/dilltest', 'a')

    def has_closure(b):
        product = a * b
        f.write("%i: %g\n" % (os.getpid(), product))
        f.flush()
        return product

    return has_closure

In [2]:
!rm -f /tmp/dilltest
closed = make_closure(5)

In [3]:
closed(2)

10

In [4]:
!cat /tmp/dilltest

25810: 10


In [5]:
import pickle

Without help, pickle can't deal with closures

In [6]:
pickle.dumps(closed)

AttributeError: Can't pickle local object 'make_closure.<locals>.has_closure'

But after we import dill, magic happens

In [7]:
import dill

In [8]:
dill.dumps(closed)[:64] + b'...'

b'\x80\x04\x95E\x03\x00\x00\x00\x00\x00\x00\x8c\ndill._dill\x94\x8c\x10_create_function\x94\x93\x94(h\x00\x8c\x0c_create_code\x94\x93...'

So from now on, pretty much everything is pickleable.

## Using dill in IPython Parallel

As usual, we start by creating our Client and View

In [9]:
import ipyparallel as ipp

cluster = ipp.Cluster(n=2)
cluster.start_cluster_sync()
rc = cluster.connect_client_sync()
rc.wait_for_engines(n=2)
view = rc.load_balanced_view()

Starting 2 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/2 [00:00<?, ?engine/s]

Now let's try sending our function with a closure:

In [10]:
view.apply_sync(closed, 3)

TypeError: cannot pickle '_io.TextIOWrapper' object

Oops, no dice. For IPython to work with dill,
there are one or two more steps. IPython will do these for you if you call `DirectView.use_dill`:

In [11]:
rc[:].use_dill()

<AsyncResult(use_dill): pending>

Now let's try again

In [12]:
view.apply_sync(closed, 3)

15

In [13]:
!cat /tmp/dilltest

25810: 10
26717: 15


Yay! Now we can use dill to allow ipyparallel to send anything.

And that's it! We can send closures, open file handles, and other previously non-pickleables to our engines.

Let's give it a try now:

In [14]:
remote_closure = view.apply_sync(make_closure, 4)
remote_closure(5)

20

In [15]:
!cat /tmp/dilltest

25810: 10
26717: 15
25810: 20


But wait, there's more!

At this point, we can send/recv all kinds of stuff

In [16]:
def outer(a):
    def inner(b):
        def inner_again(c):
            return c * b * a

        return inner_again

    return inner

So outer returns a function with a closure, which returns a function with a closure.

Now, we can resolve the first closure on the engine, the second here, and the third on a different engine,
after passing through a lambda we define here and call there, just for good measure.

In [17]:
view.apply_sync(lambda f: f(3), view.apply_sync(outer, 1)(2))

6

And for good measure, let's test that normal execution still works:

In [18]:
%%px
foo = 5

In [19]:
print(rc[:]['foo'])
rc[:]['bar'] = lambda: 2 * foo
rc[:].apply_sync(ipp.Reference('bar'))

[5, 5]


[10, 10]

And test that the `@interactive` decorator works

In [20]:
%%file testdill.py
import ipyparallel as ipp

@ipp.interactive
class C:
    a = 5

@ipp.interactive
class D(C):
    b = 10

@ipp.interactive
def foo(a):
    return a * b


Writing testdill.py


In [21]:
import testdill

In [22]:
v = rc[-1]
v['D'] = testdill.D
d = v.apply_sync(lambda: D())  # noqa: F821
print(d.a, d.b)

5 10


In [23]:
v['b'] = 10
v.apply_sync(testdill.foo, 5)

50